## LDA code setup

In [1]:

import nltk as nltk
import openpyxl as openpyxl
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import *
from sklearn.metrics.pairwise import cosine_similarity
from openpyxl import Workbook
import nltk
import pandas as pd
from numpy import array
from collections import Counter
#from gensim.models import Word2Vec

from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from numpy import s_
import datetime


In [2]:
reviews = pd.read_csv('reviews_till_Date.csv')

In [3]:
reviews.head()

,Unnamed: 0,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,0,gp:AOqpTOEyU7-40Ras_s2PEETG3BRpqcnX70XPSMadRe3...,Adebola Babatunde,https://play-lh.googleusercontent.com/a/AATXAJ...,there should be an option for a chat,3,0,1.0.11,2021-12-02 19:01:45,NaN,NaN
1,1,gp:AOqpTOE8mZw_WPC2DJbWApajZNvT1rwy3EacEG87UcR...,James maliet manyuat,https://play-lh.googleusercontent.com/a/AATXAJ...,very fantastic app,5,0,1.0.18,2021-12-02 18:36:37,NaN,NaN
2,2,gp:AOqpTOE7ocCUUOQQWaUBYc7UPcSDwBbyGF35bdXeUSa...,Tofighe Mokhtari,https://play-lh.googleusercontent.com/a-/AOh14...,Good,5,0,1.0.16,2021-12-02 18:05:22,NaN,NaN
3,3,gp:AOqpTOE4axGizzCOXlL9cKplteffne0vG_wtREXlmqZ...,mehran aslani,https://play-lh.googleusercontent.com/a-/AOh14...,🤦,2,0,1.0.16,2021-12-02 17:43:50,NaN,NaN
4,4,gp:AOqpTOFda4wLwvfII1Vo1tvZ3kic07FivwVuS4mPmax...,Thelma Smiles,https://play-lh.googleusercontent.com/a-/AOh14...,cool but always having network problems,4,0,1.0.18,2021-12-02 17:31:46,NaN,NaN


In [4]:
#changing the name of 1st column and the date column
reviews.rename(columns={reviews.columns[0]: "s_no", "at": "date_posted"}, inplace = True)


In [5]:
reviews.head()

,s_no,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,date_posted,replyContent,repliedAt
0,0,gp:AOqpTOEyU7-40Ras_s2PEETG3BRpqcnX70XPSMadRe3...,Adebola Babatunde,https://play-lh.googleusercontent.com/a/AATXAJ...,there should be an option for a chat,3,0,1.0.11,2021-12-02 19:01:45,NaN,NaN
1,1,gp:AOqpTOE8mZw_WPC2DJbWApajZNvT1rwy3EacEG87UcR...,James maliet manyuat,https://play-lh.googleusercontent.com/a/AATXAJ...,very fantastic app,5,0,1.0.18,2021-12-02 18:36:37,NaN,NaN
2,2,gp:AOqpTOE7ocCUUOQQWaUBYc7UPcSDwBbyGF35bdXeUSa...,Tofighe Mokhtari,https://play-lh.googleusercontent.com/a-/AOh14...,Good,5,0,1.0.16,2021-12-02 18:05:22,NaN,NaN
3,3,gp:AOqpTOE4axGizzCOXlL9cKplteffne0vG_wtREXlmqZ...,mehran aslani,https://play-lh.googleusercontent.com/a-/AOh14...,🤦,2,0,1.0.16,2021-12-02 17:43:50,NaN,NaN
4,4,gp:AOqpTOFda4wLwvfII1Vo1tvZ3kic07FivwVuS4mPmax...,Thelma Smiles,https://play-lh.googleusercontent.com/a-/AOh14...,cool but always having network problems,4,0,1.0.18,2021-12-02 17:31:46,NaN,NaN


In [6]:
#adjusting the date column to datetime format for filtering purpose in future
reviews["date_posted"] = pd.to_datetime(reviews["date_posted"], format='%Y-%m-%d')

In [7]:
reviews["date_posted"].head()

0   2021-12-02 19:01:45
1   2021-12-02 18:36:37
2   2021-12-02 18:05:22
3   2021-12-02 17:43:50
4   2021-12-02 17:31:46
Name: date_posted, dtype: datetime64[ns]

In [8]:
#testing the date filters
reviews.loc[reviews['date_posted'] >= "2021-12-02"]

,s_no,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,date_posted,replyContent,repliedAt
0,0,gp:AOqpTOEyU7-40Ras_s2PEETG3BRpqcnX70XPSMadRe3...,Adebola Babatunde,https://play-lh.googleusercontent.com/a/AATXAJ...,there should be an option for a chat,3,0,1.0.11,2021-12-02 19:01:45,NaN,NaN
1,1,gp:AOqpTOE8mZw_WPC2DJbWApajZNvT1rwy3EacEG87UcR...,James maliet manyuat,https://play-lh.googleusercontent.com/a/AATXAJ...,very fantastic app,5,0,1.0.18,2021-12-02 18:36:37,NaN,NaN
2,2,gp:AOqpTOE7ocCUUOQQWaUBYc7UPcSDwBbyGF35bdXeUSa...,Tofighe Mokhtari,https://play-lh.googleusercontent.com/a-/AOh14...,Good,5,0,1.0.16,2021-12-02 18:05:22,NaN,NaN
3,3,gp:AOqpTOE4axGizzCOXlL9cKplteffne0vG_wtREXlmqZ...,mehran aslani,https://play-lh.googleusercontent.com/a-/AOh14...,🤦,2,0,1.0.16,2021-12-02 17:43:50,NaN,NaN
4,4,gp:AOqpTOFda4wLwvfII1Vo1tvZ3kic07FivwVuS4mPmax...,Thelma Smiles,https://play-lh.googleusercontent.com/a-/AOh14...,cool but always having network problems,4,0,1.0.18,2021-12-02 17:31:46,NaN,NaN
5,5,gp:AOqpTOEq5oxzZAJj1D4ByLrUykIzd3Q1V3UE6Y00LZV...,Hussain Alhussain,https://play-lh.googleusercontent.com/a/AATXAJ...,𝗇𝗂𝖼𝖾 𝖺𝗉𝗉,5,0,1.0.18,2021-12-02 16:08:57,NaN,NaN
6,6,gp:AOqpTOH_UNmZbGx_WV-OGgDeM8mp4NhT_NAH_A2mbGE...,Mustapha Hadjoudja,https://play-lh.googleusercontent.com/a/AATXAJ...,Amazing,5,0,NaN,2021-12-02 15:24:01,NaN,NaN
7,7,gp:AOqpTOFBhPdyhUCw6YDVDWKvOrV0TPLfbUF4_f1s6G8...,guard Justice5,https://play-lh.googleusercontent.com/a/AATXAJ...,This app store your conversation and dont allo...,1,0,NaN,2021-12-02 15:07:08,NaN,NaN
8,8,gp:AOqpTOFWEq0aDq3v1dZVncoVMYkrne31rl-i1UTpUdZ...,David Dominic,https://play-lh.googleusercontent.com/a/AATXAJ...,harry fabrice club house is perfect,5,0,1.0.18,2021-12-02 14:57:30,NaN,NaN
9,9,gp:AOqpTOHRTaIGLof87XFpxALkCj2Nx9JafHwLjrUe9fw...,Mashallah Shamsolvaezin,https://play-lh.googleusercontent.com/a/AATXAJ...,کلاب هاوس به اینترنت قوی نیاز دارد درحالیکه وا...,2,0,1.0.16,2021-12-02 14:38:00,NaN,NaN


In [9]:

lemmatizer = nltk.stem.WordNetLemmatizer()

#Step 1: Transform reviews into term doc matrix, lemmatize, remove stop words etc
processed_reviews = []
original_reviews = []
for each_review in reviews["content"]:
    original_reviews.append(each_review)
    review_token = nltk.word_tokenize(each_review.lower())
    review_token = [lemmatizer.lemmatize(token) for token in review_token if token.isalpha()]
    review_token = [token for token in review_token if not token in stopwords.words('english')]
    processed_reviews.append(review_token)


In [10]:
print(processed_reviews[3])

[]


In [11]:
original_reviews[0:4]

['there should be an option for a chat', 'very fantastic app', 'Good', '🤦']

In [12]:
#tf-idf
Tfidf_input = []
for i in processed_reviews:
    Tfidf_input.append(" ".join(i))

#applying unigram and bigram and keeping term doc frequency = 5
review_vect = TfidfVectorizer(ngram_range=(1,2), min_df=4)
review_vect.fit(Tfidf_input)
review_vect2 = review_vect.transform(Tfidf_input)


In [13]:
print(review_vect2.toarray())
print(review_vect2.toarray().shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(29325, 6371)


In [14]:
#Step 2: Use LDA model to extract topics of each document
terms = review_vect.get_feature_names()
lda = LatentDirichletAllocation(n_components=10).fit(review_vect2)

topic_dict = {}
for topic_idx, topic in enumerate(lda.components_):
    topic_dict[topic_idx] = ",".join([terms[i] for i in topic.argsort()[-10:]])
    print("Topic %d:" % (topic_idx))
    print(",".join([terms[i] for i in topic.argsort()[:-10-1:-1]]))

Topic 0:
love,worst,like,app,cool,useful,worst app,people,voice,idea
Topic 1:
working,superb,gud,عالی,app,loved,entertainment,nyc,app working,like
Topic 2:
app,excellent,great,great app,social,experience,medium,social medium,new,platform
Topic 3:
awesome,account,app,still,get,invite,use,club,number,day
Topic 4:
good,good app,app,change,name,one,change name,good one,good application,username
Topic 5:
great,nice app,app,nice,perfect,waste,time,wonderful,waste time,user
Topic 6:
nice,best,open,best app,interesting,ca,app,doe,work,sign
Topic 7:
ok,option,add,dark,update,mode,please,need,dark mode,sound
Topic 8:
bad,wow,error,fantastic,audio,app,full,platform,better,bad app
Topic 9:
super,amazing,room,app,bug,glitch,poor,many,amazing app,connection


In [21]:
# positive reviews and first 3 months
reviews_f3_positive = reviews.loc[reviews['date_posted'] >= "2021-05-01"]
reviews_f3_positive = reviews_f3_positive.loc[reviews_f3_positive['date_posted'] <= "2021-07-31"]
reviews_f3_positive = reviews_f3_positive.loc[reviews_f3_positive['score'] >= 3]
reviews_f3_positive = reviews_f3_positive.loc[reviews_f3_positive['thumbsUpCount'] > 0]
reviews_f3_positive.head()

,s_no,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,date_posted,replyContent,repliedAt
6830,6830,gp:AOqpTOH9f-pptgr9AdH9_fTNRMCZOzYSnDCbHk3xxs3...,MUKESH KUMAR MAHAKUD,https://play-lh.googleusercontent.com/a-/AOh14...,The APP is good but it has lots of Bugs. The A...,4,21,1.0.2,2021-07-30 21:45:11,NaN,NaN
6841,6841,gp:AOqpTOGLVHXadc2yjD20DZJaT5FC7eWQ4lSc3MaQKjZ...,asiegbu victory,https://play-lh.googleusercontent.com/a/AATXAJ...,Clubhouse itself is interesting. I don't know ...,3,17,1.0.2,2021-07-30 16:47:07,NaN,NaN
6852,6852,gp:AOqpTOHVSc-2niRSr0WSbUXCdGlegh5HAYyS608pc6M...,Evan Thomas,https://play-lh.googleusercontent.com/a/AATXAJ...,It takes a while to get going and to connect w...,5,4,1.0.2,2021-07-30 14:29:46,NaN,NaN
6858,6858,gp:AOqpTOGaibWaxoXKw5L-G9aDei-c9CSUMywT8kVPchp...,Arfan Warsame,https://play-lh.googleusercontent.com/a-/AOh14...,Amazing app and I certainly do enjoy the exper...,4,5,1.0.2,2021-07-30 13:55:28,NaN,NaN
6860,6860,gp:AOqpTOH9ETfzJkSaQL2mJIUrjCJcZA-I48sSHBHWGIN...,Nikisticlife,https://play-lh.googleusercontent.com/a-/AOh14...,It would be a really great experience of us us...,4,2,1.0.1,2021-07-30 13:37:59,NaN,NaN


In [22]:
reviews_f3_positive.tail()

,s_no,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,date_posted,replyContent,repliedAt
29302,29302,gp:AOqpTOGEGnCoWc0PmkmKO1CYD0oiVcVdHBlOiMuN24y...,yousef ga0mer,https://play-lh.googleusercontent.com/a-/AOh14...,روعه,5,1,NaN,2021-05-20 19:44:13,NaN,NaN
29309,29309,gp:AOqpTOG9tNva76Zq-ECJbYYmU8UGyb2Cxx9_5nBSUBG...,مذكرات إنسان مُعاصر,https://play-lh.googleusercontent.com/a-/AOh14...,we are the top 10,5,1,NaN,2021-05-20 19:43:04,NaN,NaN
29320,29320,gp:AOqpTOFIq7t_2Jvx3J40xTt7arRe3IGbCqiyBzHE8JD...,mmd_ saffari,https://play-lh.googleusercontent.com/a-/AOh14...,😐,5,1,0.1.3,2021-05-20 19:38:46,NaN,NaN
29323,29323,gp:AOqpTOF9tKVFWuHuBfXDQmJIGYxSyRZUbMZ1SbcJ4NW...,sam bauer,https://play-lh.googleusercontent.com/a-/AOh14...,first review 🤣👍,5,7,0.1.3,2021-05-20 19:35:23,NaN,NaN
29324,29324,gp:AOqpTOH5arMLueuSroZeZaw5qQvSVeqg9GaE4QXXc9Z...,David Carr,https://play-lh.googleusercontent.com/a-/AOh14...,So excited to have Clubhouse on Android!,5,3,NaN,2021-05-20 19:28:51,NaN,NaN


In [23]:
lemmatizer = nltk.stem.WordNetLemmatizer()

#Step 1: Transform reviews into term doc matrix, lemmatize, remove stop words etc
processed_reviews = []
original_reviews = []
for each_review in reviews_f3_positive["content"]:
    original_reviews.append(each_review)
    review_token = nltk.word_tokenize(each_review.lower())
    review_token = [lemmatizer.lemmatize(token) for token in review_token if token.isalpha()]
    review_token = [token for token in review_token if not token in stopwords.words('english')]
    processed_reviews.append(review_token)

    
#tf-idf
Tfidf_input = []
for i in processed_reviews:
    Tfidf_input.append(" ".join(i))

#applying unigram and bigram and keeping term doc frequency = 5
review_vect = TfidfVectorizer(ngram_range=(1,2), min_df=4)
review_vect.fit(Tfidf_input)
review_vect2 = review_vect.transform(Tfidf_input)


#Step 2: Use LDA model to extract topics of each document
terms = review_vect.get_feature_names()
lda = LatentDirichletAllocation(n_components=10).fit(review_vect2)

topic_dict = {}
for topic_idx, topic in enumerate(lda.components_):
    topic_dict[topic_idx] = ",".join([terms[i] for i in topic.argsort()[-10:]])
    print("Topic %d:" % (topic_idx))
    print(",".join([terms[i] for i in topic.argsort()[:-10-1:-1]]))

Topic 0:
like,app,people,feature,room,work,version,option,speaker,invite
Topic 1:
nice,app,voice,chat,quality,voice quality,love,nice app,really,though
Topic 2:
good,platform,error,good platform,great platform,great,app,wa,code,join
Topic 3:
please fix,please,platform,fix,great platform,great,discus,people,help,topic
Topic 4:
app,data,great,great app,high,bug,network issue,usage,crash,thing
Topic 5:
people,app,available,room,guy,feature,good,finally,io,great
Topic 6:
audio,app,dark,mode,great,good,lot,dark mode,issue,need
Topic 7:
phone,give,private,privacy,need,android phone,bit,thank,still,hand
Topic 8:
social,ok,social medium,medium,account,worst,platform,app,useful,much
Topic 9:
instagram,account,instagram account,twitter,app,clubhouse,link,android,add,room


In [24]:
# negative reviews and first 3 months
reviews_f3_negative = reviews.loc[reviews['date_posted'] >= "2021-05-01"]
reviews_f3_negative = reviews_f3_negative.loc[reviews_f3_negative['date_posted'] <= "2021-07-31"]
reviews_f3_negative = reviews_f3_negative.loc[reviews_f3_negative['score'] <= 2]
reviews_f3_negative = reviews_f3_negative.loc[reviews_f3_negative['thumbsUpCount'] > 0]
reviews_f3_negative.head()

,s_no,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,date_posted,replyContent,repliedAt
6866,6866,gp:AOqpTOErK_eGyBqRbaRIsTp5G0G2bx1zupNih1B5z4J...,sahad zaheer,https://play-lh.googleusercontent.com/a-/AOh14...,I am not able to use it as notifications are s...,1,40,1.0.2,2021-07-30 13:11:39,NaN,NaN
6908,6908,gp:AOqpTOHfQxUXi1BMyQmdd3aNTuZgzPs5rEqQniHWW1J...,Praffulkrishna Yuvaraj,https://play-lh.googleusercontent.com/a-/AOh14...,Using a lot of mobile data to interact so that...,1,2,1.0.2,2021-07-30 05:33:40,NaN,NaN
6956,6956,gp:AOqpTOEuHwJzJdSuu-qHorFH4ljJSftOIlLG2JKuGCQ...,Quintessence Enderlyne,https://play-lh.googleusercontent.com/a-/AOh14...,I keep receiving a 6 digit verification code w...,1,1,1.0.2,2021-07-29 18:27:19,NaN,NaN
6961,6961,gp:AOqpTOGbLj1YQf9ezBpaTX5wBkHcSEaAVWkfRxO0C6U...,Fa Rah,https://play-lh.googleusercontent.com/a-/AOh14...,The app is good for conversation but it has ve...,2,5,1.0.2,2021-07-29 17:09:23,NaN,NaN
6968,6968,gp:AOqpTOFwa2Xp09o_PCMwVYiMiQiJaOxvOFYYZYgB-V-...,Rajesh kumar,https://play-lh.googleusercontent.com/a-/AOh14...,I'm using one plus mobile with 5G network but ...,2,4,1.0.2,2021-07-29 16:11:24,NaN,NaN


In [25]:
reviews_f3_negative.tail()

,s_no,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,date_posted,replyContent,repliedAt
29260,29260,gp:AOqpTOHiMm656ZsBE7ulpoF8v2trr6X4Vqsov8Lbzzq...,Joe Yakout,https://play-lh.googleusercontent.com/a-/AOh14...,Already saying number isn't supported after fa...,1,14,0.1.3,2021-05-20 19:55:27,NaN,NaN
29262,29262,gp:AOqpTOHdEl7GON52-5ZbOl-qOii334Wvjy8hzHZi61w...,rudy ernandas,https://play-lh.googleusercontent.com/a-/AOh14...,Not recieve text code,1,3,0.1.3,2021-05-20 19:55:05,"Apologies for the issues, your Clubhouse SMS v...",2021-05-21 03:08:37
29273,29273,gp:AOqpTOHhwbVXGx4TCxImxDuIbrrBJLJiNy4R4RpOnLw...,Islam Mohamed,https://play-lh.googleusercontent.com/a-/AOh14...,"it is not opening every time i open it, it jus...",1,3,0.1.3,2021-05-20 19:50:18,NaN,NaN
29277,29277,gp:AOqpTOFO6GnZjwE4wAcrAl24A4gd1xHo3XXyquFGpVj...,Abraham Samma,https://play-lh.googleusercontent.com/a-/AOh14...,Crashed promptly on start. Can't even start th...,1,18,0.1.3,2021-05-20 19:49:22,NaN,NaN
29281,29281,gp:AOqpTOHn56uAPE-3QgbXE51YBtBIMMeYG96Ybv7mf80...,Abdallah Hesham,https://play-lh.googleusercontent.com/a/AATXAJ...,البرنامج مش بيفتح,1,2,0.1.3,2021-05-20 19:47:59,NaN,NaN


In [26]:
lemmatizer = nltk.stem.WordNetLemmatizer()

#Step 1: Transform reviews into term doc matrix, lemmatize, remove stop words etc
processed_reviews = []
original_reviews = []
for each_review in reviews_f3_negative["content"]:
    original_reviews.append(each_review)
    review_token = nltk.word_tokenize(each_review.lower())
    review_token = [lemmatizer.lemmatize(token) for token in review_token if token.isalpha()]
    review_token = [token for token in review_token if not token in stopwords.words('english')]
    processed_reviews.append(review_token)

    
#tf-idf
Tfidf_input = []
for i in processed_reviews:
    Tfidf_input.append(" ".join(i))

#applying unigram and bigram and keeping term doc frequency = 5
review_vect = TfidfVectorizer(ngram_range=(1,2), min_df=4)
review_vect.fit(Tfidf_input)
review_vect2 = review_vect.transform(Tfidf_input)


#Step 2: Use LDA model to extract topics of each document
terms = review_vect.get_feature_names()
lda = LatentDirichletAllocation(n_components=10).fit(review_vect2)

topic_dict = {}
for topic_idx, topic in enumerate(lda.components_):
    topic_dict[topic_idx] = ",".join([terms[i] for i in topic.argsort()[-10:]])
    print("Topic %d:" % (topic_idx))
    print(",".join([terms[i] for i in topic.argsort()[:-10-1:-1]]))

Topic 0:
join,account,room,error,create,unable,app,create account,showing,join room
Topic 1:
bad,network,issue,fake,network issue,app,data,receive,say,usage
Topic 2:
code,doe,work,verification,verification code,open,doe work,system,doe open,app
Topic 3:
account,ready,account ready,reserved,app,text,week,house,day,problem
Topic 4:
instagram,ever,option,app ever,instagram account,account,add,app,privacy,worst
Topic 5:
app,time,worst,worst app,waste,open,issue,waste time,even,use
Topic 6:
working,please,fix,app,error,glitch,please fix,time,even,like
Topic 7:
number,user,ca,get,phone number,phone,hear,code,invite,incorrect
Topic 8:
voice,quality,opening,come,bad,audio,waiting,sound,voice quality,text
Topic 9:
connection,poor,poor connection,bug,try,many bug,many,internet,audio,always


In [27]:
# positive reviews and last 3 months
reviews_l3_positive = reviews.loc[reviews['date_posted'] >= "2021-08-01"]
reviews_l3_positive = reviews_l3_positive.loc[reviews_l3_positive['score'] >= 3]
reviews_l3_positive = reviews_l3_positive.loc[reviews_l3_positive['thumbsUpCount'] > 0]
reviews_l3_positive.head()

,s_no,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,date_posted,replyContent,repliedAt
36,36,gp:AOqpTOHDDYkkL1AJouqSM5GDPyEpHEu_lpN75T1-K1A...,Ezinne Nwadike,https://play-lh.googleusercontent.com/a-/AOh14...,"Good app, sort of. I'd like to to be able to c...",4,3,1.0.16,2021-12-01 18:03:46,NaN,NaN
80,80,gp:AOqpTOHkZJY28dL1sCTKYFz3tksnMzy5MQobHzWuPFy...,mjeed elamen,https://play-lh.googleusercontent.com/a/AATXAJ...,Nice,5,1,0.1.9,2021-11-30 12:38:30,NaN,NaN
94,94,gp:AOqpTOER17lxDiaaLe4ZmoNpOLlUgXVTHZ6hsI5cII8...,HITESH NATHYA,https://play-lh.googleusercontent.com/a-/AOh14...,a good social app but i request to the develop...,4,3,1.0.16,2021-11-30 05:11:12,NaN,NaN
138,138,gp:AOqpTOFPkfDkdDoEDNs-Ei2Nn5Nwij5InA5q4OIIKU9...,Sagar S,https://play-lh.googleusercontent.com/a-/AOh14...,This app lets you have live audio conversation...,5,3,1.0.16,2021-11-28 15:19:57,NaN,NaN
170,170,gp:AOqpTOFQocY0TYDXRXQsgZVwjq3R6ingyOQvJBm6WnE...,Akshay Babu,https://play-lh.googleusercontent.com/a-/AOh14...,Please provide an option to speed up the repla...,4,3,1.0.16,2021-11-27 09:54:10,NaN,NaN


In [28]:
reviews_l3_positive.tail()

,s_no,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,date_posted,replyContent,repliedAt
6599,6599,gp:AOqpTOFFoZ6d8ZDA1Sf0how-BfEqz4TDV52b4y80D7u...,Kiranraj Muthuraj,https://play-lh.googleusercontent.com/a-/AOh14...,Great app for people with different people and...,5,7,1.0.2,2021-08-02 06:54:15,NaN,NaN
6620,6620,gp:AOqpTOHD67hi2f0fzTq7Ln2y8SvP3LrSoMIs5CgGjEZ...,Khaled Reda,https://play-lh.googleusercontent.com/a-/AOh14...,There's a problem with the Bluetooth headset w...,3,2,1.0.2,2021-08-02 01:16:48,NaN,NaN
6633,6633,gp:AOqpTOF7Y3hONniJ43mFkXNAihu9r6MIakssDsAHhZu...,Ally Bukuku,https://play-lh.googleusercontent.com/a/AATXAJ...,It's like being in a conference center where t...,5,1,1.0.2,2021-08-01 16:51:24,NaN,NaN
6639,6639,gp:AOqpTOELLyux_9168aYzThlkpzMloHWanRmy14QwTtq...,ali mohammadi,https://play-lh.googleusercontent.com/a-/AOh14...,I entered Club House a month ago through the w...,5,2,1.0.2,2021-08-01 15:48:33,NaN,NaN
6670,6670,gp:AOqpTOF6zPq-1Wxr1_3-3COpW3baTCmkqClHHpMVghk...,Shashank Hegade,https://play-lh.googleusercontent.com/a-/AOh14...,"One request...When playing in background, It s...",4,2,1.0.1,2021-08-01 09:56:26,NaN,NaN


In [29]:
lemmatizer = nltk.stem.WordNetLemmatizer()

#Step 1: Transform reviews into term doc matrix, lemmatize, remove stop words etc
processed_reviews = []
original_reviews = []
for each_review in reviews_l3_positive["content"]:
    original_reviews.append(each_review)
    review_token = nltk.word_tokenize(each_review.lower())
    review_token = [lemmatizer.lemmatize(token) for token in review_token if token.isalpha()]
    review_token = [token for token in review_token if not token in stopwords.words('english')]
    processed_reviews.append(review_token)

    
#tf-idf
Tfidf_input = []
for i in processed_reviews:
    Tfidf_input.append(" ".join(i))

#applying unigram and bigram and keeping term doc frequency = 5
review_vect = TfidfVectorizer(ngram_range=(1,2), min_df=4)
review_vect.fit(Tfidf_input)
review_vect2 = review_vect.transform(Tfidf_input)


#Step 2: Use LDA model to extract topics of each document
terms = review_vect.get_feature_names()
lda = LatentDirichletAllocation(n_components=10).fit(review_vect2)

topic_dict = {}
for topic_idx, topic in enumerate(lda.components_):
    topic_dict[topic_idx] = ",".join([terms[i] for i in topic.argsort()[-10:]])
    print("Topic %d:" % (topic_idx))
    print(",".join([terms[i] for i in topic.argsort()[:-10-1:-1]]))

Topic 0:
mode,dark,dark mode,wa,error,room,still,bad,try,lot
Topic 1:
network,world,audio quality,audio,voice,much,theme,quality,improvement,need
Topic 2:
time,app,battery,annoying,night,eye,love,day,take,clubhouse
Topic 3:
nice,see,one,room,app,please,way,problem,go,clubhouse
Topic 4:
platform,app,login,best,great,room,different,like,social,time
Topic 5:
great,please,app,number,show,talk,bug,moderator,bluetooth,much
Topic 6:
good,good app,app,perfect,quality,idea,sound,support,user,best
Topic 7:
application,topic,really,new,people,communication,many,new people,meeting,like
Topic 8:
like,app,chat,option,good,bad,must,clear,connect people,connect
Topic 9:
option,message,speaker,delete,add,room,app,please,back,great


In [30]:
# negative reviews and last 3 months
reviews_l3_negative = reviews.loc[reviews['date_posted'] >= "2021-08-01"]
reviews_l3_negative = reviews_l3_negative.loc[reviews_l3_negative['score'] <= 2]
reviews_l3_negative = reviews_l3_negative.loc[reviews_l3_negative['thumbsUpCount'] > 0]
reviews_l3_negative.head()

,s_no,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,date_posted,replyContent,repliedAt
81,81,gp:AOqpTOEVBVbTPkTmRXaz0P_KTRVbVgfUhmMNbENX_L_...,Joseph Weeks,https://play-lh.googleusercontent.com/a/AATXAJ...,"I used to enjoy this app, really got allot out...",1,8,1.0.16,2021-11-30 12:04:29,NaN,NaN
85,85,gp:AOqpTOF_Zia8ubPVlgpWHIisCQuPbfs4b0JmDdHDiWB...,Mad Professional,https://play-lh.googleusercontent.com/a-/AOh14...,Needs some work .. 1. a hand raising options o...,2,6,1.0.16,2021-11-30 09:57:15,NaN,NaN
113,113,gp:AOqpTOET0pXXun8TIxn2OWeJebdDNbayeOZ8Wx--CkP...,JOSEF ANTONY,https://play-lh.googleusercontent.com/a-/AOh14...,Daily error new bug ..now i cant login..waitin...,1,2,1.0.16,2021-11-29 15:42:22,NaN,NaN
135,135,gp:AOqpTOE0ifRX1yneNtt_HCZjptZMUVb3YVVdhLotatU...,xx,https://play-lh.googleusercontent.com/a-/AOh14...,"I can't get in. It keeps saying ""error..try ag...",1,1,1.0.16,2021-11-28 21:02:50,NaN,NaN
145,145,gp:AOqpTOGO_GmjL2oM02JkBkCsFKgpl86WxpUEO-rxuoa...,Santhosh,https://play-lh.googleusercontent.com/a-/AOh14...,I can't choose interests what i want ...Becaus...,1,2,1.0.16,2021-11-28 11:36:04,NaN,NaN


In [31]:
reviews_l3_negative.tail()

,s_no,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,date_posted,replyContent,repliedAt
6461,6461,gp:AOqpTOFB01C2WyuvXOn_RbsQN-96aC7sN-E6-3YGZF5...,Ibrahim Siddiq,https://play-lh.googleusercontent.com/a-/AOh14...,04 August 2021 Why can't I login to my Clubhou...,2,230,1.0.2,2021-08-03 21:07:07,NaN,NaN
6493,6493,gp:AOqpTOFfnDaJ44m-GOIzvt8DeYvrDiKbw2PWFQQrifX...,Amit Deshmukh,https://play-lh.googleusercontent.com/a-/AOh14...,App is not useful some good but many timpasser...,1,2,1.0.2,2021-08-03 11:02:11,NaN,NaN
6508,6508,gp:AOqpTOGGmd9AIyhFY3fPnOeeWz9aJ1t-RVSTKQ4Kscy...,Adnan Al Falasi,https://play-lh.googleusercontent.com/a-/AOh14...,poor connection and slow re-connect .... but s...,2,3,1.0.2,2021-08-03 07:51:48,NaN,NaN
6583,6583,gp:AOqpTOE-E2YCCiN5M3PULb8DnjUxKUJrcWZlVICWyMB...,narigi anitha,https://play-lh.googleusercontent.com/a/AATXAJ...,Worst experience tried to login many times but...,1,1,1.0.0,2021-08-02 11:49:46,NaN,NaN
6637,6637,gp:AOqpTOFSy5H4Ntm9eBsfN_GmpJOBft5BT1TemdZA92S...,Tarek Adel,https://play-lh.googleusercontent.com/a-/AOh14...,How to contact the app support team? Suddenly ...,1,8,1.0.2,2021-08-01 15:56:34,NaN,NaN


In [32]:
lemmatizer = nltk.stem.WordNetLemmatizer()

#Step 1: Transform reviews into term doc matrix, lemmatize, remove stop words etc
processed_reviews = []
original_reviews = []
for each_review in reviews_l3_negative["content"]:
    original_reviews.append(each_review)
    review_token = nltk.word_tokenize(each_review.lower())
    review_token = [lemmatizer.lemmatize(token) for token in review_token if token.isalpha()]
    review_token = [token for token in review_token if not token in stopwords.words('english')]
    processed_reviews.append(review_token)

    
#tf-idf
Tfidf_input = []
for i in processed_reviews:
    Tfidf_input.append(" ".join(i))

#applying unigram and bigram and keeping term doc frequency = 5
review_vect = TfidfVectorizer(ngram_range=(1,2), min_df=4)
review_vect.fit(Tfidf_input)
review_vect2 = review_vect.transform(Tfidf_input)


#Step 2: Use LDA model to extract topics of each document
terms = review_vect.get_feature_names()
lda = LatentDirichletAllocation(n_components=10).fit(review_vect2)

topic_dict = {}
for topic_idx, topic in enumerate(lda.components_):
    topic_dict[topic_idx] = ",".join([terms[i] for i in topic.argsort()[-10:]])
    print("Topic %d:" % (topic_idx))
    print(",".join([terms[i] for i in topic.argsort()[:-10-1:-1]]))

Topic 0:
channel,back,speaker,back channel,glitch,concept,network issue,many,facing,issue
Topic 1:
full,bad,follower,want,account,bug,month,work,someone,mean
Topic 2:
adult,let,group,joining,room,problem,great,wa,application,month
Topic 3:
connection,poor,room,poor connection,update,wifi,app,showing,always,listen
Topic 4:
log,chat,room,delete,user,message,even,change,app,send
Topic 5:
error,try,login,wa error,ca,account,please try,wa,please,error please
Topic 6:
app,room,audio,group,people,thing,sex,ca,lot,talk
Topic 7:
number,app,phone,phone number,call,code,verification,fix,device,version
Topic 8:
remove,language,room,interest,message,two,need,section,mode,text
Topic 9:
app,time,voice,experience,sexual,bad,audio,use,room,bluetooth


In [122]:
terms = review_vect.get_feature_names()
terms

['aa',
 'aap',
 'aap hai',
 'aapp',
 'aaya',
 'ab',
 'abhi',
 'ability',
 'ability change',
 'able',
 'able access',
 'able add',
 'able change',
 'able connect',
 'able create',
 'able delete',
 'able download',
 'able enter',
 'able follow',
 'able get',
 'able hear',
 'able invite',
 'able join',
 'able link',
 'able listen',
 'able log',
 'able login',
 'able make',
 'able open',
 'able participate',
 'able register',
 'able see',
 'able sign',
 'able talk',
 'able update',
 'able use',
 'abruptly',
 'absolute',
 'absolutely',
 'absolutely awesome',
 'absolutely love',
 'absurd',
 'abt',
 'abuse',
 'abusing',
 'abusive',
 'abusive language',
 'ac',
 'acc',
 'acc ready',
 'accept',
 'acceptable',
 'accepted',
 'accepting',
 'access',
 'access account',
 'access app',
 'access contact',
 'accessibility',
 'accessible',
 'accessing',
 'accidentally',
 'accont',
 'according',
 'according interest',
 'accordingly',
 'account',
 'account able',
 'account account',
 'account activated',
 